<a href="https://colab.research.google.com/github/edmenciab733/qosf/blob/main/Optimizaci%C3%B3n_de_Numero_de_Personales_en_un_call_center.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Optimización de cantidad de personal en un callcenter utilizando

# Librerias

In [ ]:
%%capture
!pip install qiskit
!pip install matplotlib
!pip install pylatexenc
!pip install pulp

# Descripción del problema

Encontrar el nro óptimo de agentes y supervisores, teniendo el cuenta el presupuesto y en un máximo de horas trabajadas. La idea es maximizar el nro de mensajes prespuestos teniendo en cuenta un promedio de mensajes recepcionados por minutos


## Función Objetivo



* Variable binaria que vale 1 si el agente i es contratado y 0 si no.: $$ x_i  $$
* Variable binaria que vale 1 si el agente i es contratado y 0 si no.: $$ y_i  $$
* Tiempo aceptable de atención por cada mensaje por el agente $$ t_a $$
* Cantidad de mensajes por minutos $$ w_a $$
* Cantidad de mensajes por minutos $$ w_s = \frac{w_a}{ta} $$
* Tiempo aceptable de atención por cada mensaje por el supervisión $$ t_s = ws $$
* Cantidad de Agentes $$ N $$
* Cantidad de Supervisores $$ M $$



Maximizar:

$$ Z = (t_a * w_a * \sum_{i=1}^N x_i)  + (t_s * w_s * \sum_{i=1}^M y_j)  $$




Solver: https://www.gams.com/latest/docs/S_CBC.html

In [ ]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpStatus

In [ ]:
import pulp as pl
solver_list = pl.listSolvers(onlyAvailable=True)
print(solver_list)

['PULP_CBC_CMD']


In [ ]:
# Crear el problema de programación lineal
model = LpProblem(name="numero-optimo-de-empleados", sense=LpMaximize)
# Definir las variables
# 5 agentes y 3 supervisores
agentes = [LpVariable(name=f"agente_{i}", cat='Binary') for i in range(5)]
supervisores = [LpVariable(name=f"supervisor_{i}", cat='Binary') for i in range(3)]



# Tasa de mensajes por minuto y tiempo de atención por mensaje
wa = 100 # cantidad de mensajes por minutos
ta = 4  # tiempo de atención en minutos


ws = round(wa /ta, 0)
ts = ws  # en minutos

# Añadir la función objetivo
model += lpSum(wa * ta * (a) for a in agentes) + lpSum(ws * ts * s for s in supervisores), "Maximizar_Atencion"


## Restricciones



### Presupuesto



*   Costo de cada agente por hora $$ cx_i $$
*   Costo de cada supervisor por hora $$ cy_i $$



$$  ( \sum_{i=1}^N x_i * cx_i)  + ( \sum_{i=1}^M y_j * cy_i) \leq presupuesto $$

In [ ]:
# Costos individuales para agentes y supervisores
agent_costs = [3000, 3200, 3100, 2900, 3050 ]
supervisor_costs = [5000, 5100, 5200]

# Presupuesto total
presupuesto = int(input("Presupuesto por día: " ))


# Añadir la restricción de presupuesto
model += lpSum(a * c for a, c in zip(agentes, agent_costs)) + lpSum(s * c for s, c in zip(supervisores, supervisor_costs)) <= presupuesto, "Presupuesto"

model += lpSum(supervisores) >= 1, "Al_menos_un_supervisor"

Presupuesto por día: 15000


### Al menos n agentes, m supervisor y relaciones

* Al menos n agentes
$$ \sum_{i=1}^N x_i \geq n $$

* Al menos m supervisores
$$ \sum_{j=1}^M y_i \geq m $$


* Relación agente supervisor 10:1



$$ 10 * \sum_{j=1}^M y_i \geq \sum_{i=1}^N x_i $$


In [ ]:
n = 3
m = 1
# Añadir la restricción de presupuesto
model += lpSum(agentes) >= n, "Al_menos_n_agente"
model += lpSum(supervisores) >= m, "Al_menos_m_supervisor"
model += 10 * lpSum(supervisores) >= lpSum(agentes), "Relacion_supervisor_agente"

# Solver

In [ ]:
model.solve()
print(f"Estado: {model.status}, {LpStatus[model.status]}")
for var in agentes + supervisores:5
    print(f"{var.name}: {var.varValue}")

print(f"Función objetivo = {model.objective.value()}")

Estado: 1, Optimal
agente_0: 1.0
agente_1: 0.0
agente_2: 1.0
agente_3: 0.0
agente_4: 1.0
supervisor_0: 1.0
supervisor_1: 0.0
supervisor_2: 0.0
Función objetivo = 1825.0
